# 4 Systems of Differential Equations

In this tutorial, two *implicit* timestepping methods are introduced, the implicit Euler and the Crank-Nicolson schemes. Once you are familiar with them, we will apply them to systems of first-order differential equations. We will also transform higher-order differential equations into systems of first-order equations, and analyse them as such. Of course they can also be applied two regular differntial equations, and the methods from the last tutorial could be applied to systems of differential equations too.

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import rc
rc("text", usetex=True) # Latex font in figures

### Implicit Euler

Consider the exponential function $y(t) = e^{-\lambda T}$, which satisfies $y'(t) = -\lambda y(t)$. Simulating it with the explicit Euler method would yield the following timestepping scheme:

$y(t+\delta) \approx y(t) -\lambda y(t) = (1-\delta\lambda)y(t)$

which creates ocillations if $\delta\lambda>1$ (first-order difference equation with negative eigenvalue). This only happens due to the discretisation of an otherwise continuous system, and is hence undesirable. A solution is the implicit Euler method:

$\frac{y(t+\delta) - y(t)}{\delta} \approx y'(t+\delta) = f(y(t+\delta),t+\delta)$

$\Leftrightarrow y(t+\delta) \approx y(t) + \delta f(y(t+\delta), t+\delta)$

I.e. we move along the tangent slope of the next point instead of the current. The actual timestepping scheme is given implicitly as the solution to a fixed-point problem. In the example of the exponential function above:

$y(t+\delta) = y(t) -\lambda y(t) = (1-\delta\lambda)y(t+\delta)$

$\Leftrightarrow y(t+\delta) = \frac{1}{1+\delta\lambda}y(t)$

*(Note the analogy to forward and backward solutions of exogenous sequences using lag polynomials).*

### Crank-Nicolson

As you noticed in the last tutorial, the explicit Euler method creates a bias if the second derivative is non-zero. The same is true for the implicit method, but the bias is in the opposite direction. A straightforward solution is to take the mean of both, so that the biases cancel each other out (completely, if no effects of orders higher than two are present):


$y(t+\delta) \approx y(t) + \frac{\delta}{2}\left[f\left(y\left(t\right), t\right) + f\left(y\left(t+\delta\right), t+\delta\right)\right]$ 

As the values at the next time step are on both sides, this is also an implicit scheme that requires finding the solution to a fixed-point equation.

##### EXERCISE

Before moving on to systems of equations, complete the functions for a single step using the implicit Euler and Crank-Nicolson methods below. The model is the continuous-time cobweb model again: $p'=45 - 3p$.

##### SOLUTION

The timestepping scheme according to the **implicit Euler** method is

$p_{t+\delta} = p_t + \delta p'_{t+\delta} = p_t + \delta (45 - 3p_{t+\delta})$

Which we need to solve for $p_{t+\delta}$:

$p_{t+\delta} = \frac{1}{1+3\delta}(p_t + 45\delta)$

For the **Crank-Nicolson method**, we have to solve $p_{t+\delta} = p_t + \frac{\delta}{2}(p'_t + p'_{t+\delta})$ for $p_{t+\delta}$. In the case of the Cobweb model:

$p_{t+\delta} = (1 - \frac{3\delta}{2})p_t - \frac{3\delta}{2}p_{t+\delta} + 45\delta$

$\Leftrightarrow (1+\frac{3\delta}{2})p_{t+\delta} = \frac{2+3\delta}{2}p_{t+\delta} = \frac{2-3\delta}{2}p_t + 45\delta$

$\Leftrightarrow p_{t+\delta} = \frac{2-3\delta}{2+3\delta}p_t + \frac{90}{2+3\delta}\delta$

In [ ]:
def imp_euler_1step(p, delta):
    # TO DO

In [ ]:
def cn_1step(p, delta):
    # TO DO

Solutions from last tutorial's methods for comparison

In [ ]:
def exp_euler_1step(p, delta):
    return p + delta * (45 - 3*p)

def runge_1step(p, delta):
    p_mid = exp_euler_1step(p, delta*0.5)
    # derivative at midpoint
    dp = 45 - 3 * p_mid
    # return current price + delta times rate of change
    return p + delta * dp

# analytical solution
def cobweb_1period(p_0, T):
    return (p_0 - 15) * np.exp(-3*T) + 15

In [ ]:
# Simulation until point T
def simulate_cobweb(p, T, n, timestep_func):
    delta = 1/n
    for t in range(n*T):
        p = timestep_func(p, delta)
    return p

In [ ]:
p0 = 60
T = 1
p_analytical = cobweb_1period(p0, T)

# errors
ee_error = ie_error = rg_error = cn_error = 1

# analyse convergence, doubling the number of steps per time unit
for exponent in range(2,6):
    n = 2 ** exponent
    # update last period errors
    ee_last_error = ee_error
    ie_last_error = ie_error
    rg_last_error = rg_error
    cn_last_error = cn_error
    
    # starting prices
    p_ee = p_ie = p_rg = p_cn = p0
    
    # calculate solutions
    p_ee = simulate_cobweb(p_ee, T, n, exp_euler_1step)
    p_ie = simulate_cobweb(p_ie, T, n, imp_euler_1step)
    p_rg = simulate_cobweb(p_rg, T, n, runge_1step)
    p_cn = simulate_cobweb(p_cn, T, n, cn_1step)
    
    # new errors
    ee_error = p_ee - p_analytical
    ie_error = p_ie - p_analytical
    rg_error = p_rg - p_analytical
    cn_error = p_cn - p_analytical
    
    # convergence
    print(f'delta = 1/{n}:')
    print(
        f'Explicit Euler scheme \n current value: p = {p_ee:.3e};\t error = {ee_error:.3e};\t error ratio = {ee_last_error / ee_error:.2e}'
    )
    print(
        f'Implicit Euler scheme \n current value: p = {p_ie:.3e};\t error = {ie_error:.3e};\t error ratio = {ie_last_error / ie_error:.2e}'
    )
    print(
        f'Central difference scheme \n current value: p = {p_rg:.3e};\t error = {rg_error:.3e};\t error ratio = {rg_last_error / rg_error:.2e}'
    )
    print(
        f'Crank-Nicolson scheme \n current value: p = {p_cn:.3e};\t error = {cn_error:.3e};\t error ratio = {cn_last_error / cn_error:.2e}'
    )
    print('\n')

As you can see, the Crank-Nicolson methods also exhibits second-order convergence, i.e. the errors are 4 times larger if we double $\delta$. The implicit Euler method, following the same reasoning as its explicit counterpart, converges proportionally to $\delta$ (first-order convergence).

For $n=2$, we would obtain the explicit Euler time-stepping scheme $p(t+\delta) = (1-3\delta)p(t) + 45\delta = -0.5p(t) + 22.5$, i.e. a first-order difference equation with a negative eigenvalue. The implicit method on the other hand yields $p_{t+\delta} = \frac{1}{1+3\delta}(p_t + 45\delta) = \frac{2}{5}(p_t + 45\delta)$, which is a difference equation with a positive eigenvalue and therefore does not lead to the undesired oscillations. 

### Systems of differential equations

Now that we know the main methods, we can apply them to systems of first-order difference equations. This step is analogous to the same extension of difference equations. We simply deal with equations in vectors and matrices instead of scalars, but everything else works in exactly the same way. 

### Two goods model

Consider the model in Tutorial 4, exercise 1:

$\begin{align}p_1' &= -2p_1 + 4p_2\\p_2' &= -p_1 + p_2\end{align}$

Or in vector notation:

$
    \left(\begin{matrix}
    p_1' \\ p_2'
    \end{matrix}\right) = \left(\begin{matrix} 
    -2 & 4 \\ -1 & 1
    \end{matrix}\right) \left(\begin{matrix}
    p_1 \\p_2
    \end{matrix}\right)
\Leftrightarrow 
    \mathbf{p}' = \mathbf{Ap}
$ 

We are going to implement one step of the explicit and implicit Euler schemes, as well as Runge's method and the Crank-Nicolson scheme in the functions below. Recall their implementations:

#### Explicit Euler
(In vector notation:)

$\mathbf{p}_{t+\delta} = \mathbf{p}_t + \delta\mathbf{p}'_{t}$

Which can be implement using the definition of the vector of derivatives above:

$\Rightarrow \mathbf{p}_{t+\delta} = \mathbf{p}_t + \delta\mathbf{Ap}_{t}$

#### Implicit Euler
$\mathbf{p}_{t+\delta} = \mathbf{p}_t + \delta\mathbf{p}'_{t+\delta} = \mathbf{p}_t + \delta\mathbf{Ap}_{t+\delta}$

This requires a finding the solution to a fixed-point problem, i.e. we have to solve for $\mathbf{p}_{t+\delta}$:

$\begin{align}
    \mathbf{p}_{t+\delta}(\mathbf{I} - \delta\mathbf{A}) &= \mathbf{p}_t \\
    \mathbf{p}_{t+\delta} &= (\mathbf{I} - \delta\mathbf{A})^{-1}\mathbf{p}_t
\end{align}$

Where $\mathbf{I}$ is the identity matrix.

#### Runge's central difference quotient
This one does not change much, it is essentially an extension of the explicit Euler scheme with an intermediate step:

$\begin{align}
    \mathbf{p}_{t+\delta} &= \mathbf{p}_t + \delta \mathbf{Ap}_{t+\delta/2}\hspace{0.1cm}; \hspace{0.5cm}\text{with}\\
    \mathbf{p}_{t+\delta/2} &= \mathbf{p}_t + \frac{\delta}{2} \mathbf{Ap}_{t}
\end{align}$


#### Crank-Nicolson
$\begin{align}
    \mathbf{p}_{t+\delta} &= \mathbf{p}_t + \frac{\delta}{2}(\mathbf{p}'_{t+\delta} + \mathbf{p}'_{t}) = (\mathbf{I} + \frac{\delta}{2}\mathbf{A})\mathbf{p}_t + \frac{\delta}{2}\mathbf{A}\mathbf{p}_{t+\delta}\\
    \Leftrightarrow
    \mathbf{p}_{t+\delta} &= (\mathbf{I}-\frac{\delta}{2}\mathbf{A})^{-1}(\mathbf{I}+\frac{\delta}{2}\mathbf{A})\mathbf{p}_t
\end{align}$


##### Exercise
Implement each of these methods in the respective functions below. They should take as inputs the current price vector $p$, the coefficient matrix $A$, and the size of the step $\delta$. Invert matrices `M` with the `inv` function from numpy's subpackage linalg: `np.linalg.inv(M)`.

In [ ]:
# explicit Euler
def ee_2goods(p, A, delta):
    # TO DO

In [ ]:
# implicit Euler
def ie_2goods(p, A, delta):
    # TO DO

In [ ]:
# Runge
def rg_2goods(p, A, delta):
    # TO DO

In [ ]:
# Crank-Nicolson
def cn_2goods(p, A, delta):
    # TO DO

##### Exercise
Implement the loop in the function below. The function simulates the behaviour of the model over time for 1 unit time step, which is divided into $1/\delta$ incremental time steps. The particular timestepping method is passed as an input as well, so later on we can exchange the methods flexibly and compare behaviour.

In [ ]:
def simulate_2d_model(p, A, T, delta, timestep_func):
    '''
    This function simulates 2-dimensional dynamic models for T unit time steps.
    
    Inputs
        p            : initial value vector (numpy array length 2)
        A            : coefficient matrix (2x2 numpy array)
        T            : number of unit steps
        delta        : size of individual time steps (<<1)
        timestep_func: a function that implements one time step increment of size delta (ee, ie, cn)

    Output
        Time series array of values, starting with the initial values
    '''
    # number of time increments
    U = int(T / delta) # to make sure it is an integer!
    
    results = np.empty((2, U+1))
    results[:,0] = p
    
    for u in range(U):
        #######
        # TO DO
        #######
        
    return results

Now we can simulate this model for a number of unit time steps and save the results to an array.

In [ ]:
T = 10 # 10 unit time steps
delta = 1/512

p = np.array([2,2]) # initial price vector

A = np.array(
    #######
    # TO DO
    #######
) # coefficient matrix

results = simulate_2d_model(p, A, T, delta, cn_2goods)

#### Plot the results

In the figure below, the results are plotted. In the left panel, you can see the time series of both prices, the right panel depicts the phase portrait.

For the phase protrait, we also have to calculate the isoclines:
    
$p_1'=0 \Leftrightarrow -2p_1 + 4 p_2 = 0 \Leftrightarrow p_2 = \frac{1}{2}p_1$

and

$p_2'=0 \Leftrightarrow -p_1 + p_2 = 0 \Leftrightarrow p_2 = p_1$

In [ ]:
x_iso = np.array([-1, 3])

p1_iso = 0.5 * x_iso
p2_iso = x_iso


fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

plt.suptitle("Samuelson's 2-goods model")
ax1.set_title("time series")
ax2.set_title("phase portrait")

x_ts = np.linspace(0, T, len(results[0]))

ax1.plot(x_ts, results[0], lw=0.7, c="darkblue", label="$p_1$")
ax1.plot(x_ts, results[1], lw=0.7, c="darkred", label="$p_2$")
ax1.hlines(0, 0, T, ls="--", lw=0.7, color="black", label="steady state")

ax1.legend()

ax2.plot(results[0], results[1], lw=0.7, c="darkgreen")
ax2.plot(x_iso, p1_iso, lw=0.7, c="black", ls='--')
ax2.plot(x_iso, p2_iso, lw=0.7, c="black", ls='--')

ax1.grid(alpha=0.4)
ax2.grid(alpha=0.4)

Find below the code for the solution figure for tutorial 4, exercise 1 b):

In [ ]:
fig = plt.figure(figsize=(6,6))

# isoclines
x_iso = np.array([-2, 3.1])
p1_iso = 0.5 * x_iso
p2_iso = x_iso

# plot dynamics and isoclines
plt.plot(results[0], results[1], lw=0.8, c="black")
plt.plot(x_iso, p1_iso, lw=0.7, c="black", ls='--')
plt.plot(x_iso, p2_iso, lw=0.7, c="black", ls='--')

# isocline annotation
plt.text(-1.6, -1.8, "$p_2'=0$")
plt.text(-1.95, -0.7, "$p_1'=0$")

# axis labels
plt.xlabel("$p_1$", loc="right", labelpad=-5, fontsize=12)
plt.ylabel("$p_2$", loc="top", labelpad=-5, fontsize=12)

# hardcode the limits of the figure
plt.xlim(-2.2, 3.25)
plt.ylim(-2.2, 3.25)

# give the figure a more "mathy" feel by removing the top and left spines, 
# add arrowheads to the others
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.plot(3.3, 0, ">k", transform=ax.get_xaxis_transform(), clip_on=False)
ax.plot(0, 3.3, "^k", transform=ax.get_yaxis_transform(), clip_on=False)

##################################
## arrows of motion
# arrows take as input the starting point and the change in both dimension

# bottom right quadrant
plt.arrow(2.5, -0.5, -0.4, 0, lw=0.5, head_width=0.075, color='black')
plt.arrow(2.5, -0.5, 0, -0.4, lw=0.5, head_width=0.075, color='black')

# bottom left quadrant
plt.arrow(-1.55, -1.45, -0.4, 0, lw=0.5, head_width=0.075, color='black')
plt.arrow(-1.55, -1.45, 0, 0.4, lw=0.5, head_width=0.075, color='black')

# top left quadrant
plt.arrow(-1.5, 1.5, 0.4, 0, lw=0.5, head_width=0.075, color='black')
plt.arrow(-1.5, 1.5, 0, 0.4, lw=0.5, head_width=0.075, color='black')

# top right quadrant
plt.arrow(2.7, 2.5, 0.4, 0, lw=0.5, head_width=0.075, color='black')
plt.arrow(2.7, 2.5, 0, -0.4, lw=0.5, head_width=0.075, color='black')

# add a grid
plt.grid(linestyle=':')

# save the figure if you want to
#plt.savefig("samuelson2goods.png")

##### Exercise

Now consider the second-order differential equation $y'' + 2y = 0$. Instead of extending the methods we have learned to higher-order systems, re-state this equation as a 2-dimensional system of first-order differential equations and simulate it. You only have to define the coefficient matrix $A$, then we can re-use the single-step functions above. Finally, we will compare the three methods with this particular equation.

##### Solution

We define $z = y'$, so that we can write the equation as the following system:

$\begin{align}
    y' &= z\\
    y'' = z' &= -2y
\end{align}$

Or, in vector notation:

$
    \left(\begin{matrix}
    y' \\ z'
    \end{matrix}\right) = \left(\begin{matrix} 
    0 & 1 \\ -2 & 0
    \end{matrix}\right) \left(\begin{matrix}
    y \\ z
    \end{matrix}\right)
$

You can confirm that this system has purely imaginary eigenvalues $\lambda_{1,2} = \sqrt{2}i$. As the real part $Re(\lambda_{1,2})=0$, it is neither converging nor diverging, but remains on a stable cycle around the equilibrium, maintaining the same distance from it at all times.

In [ ]:
A = np.array(
    #######
    # TO DO
    #######
)

y0 = np.array([2,2])

T = 20
delta = 1/32

# we can apply the 2-goods model function, as the steps are exactly the same!
results_ee = simulate_2d_model(y0, A, T, delta, ee_2goods)
results_ie = simulate_2d_model(y0, A, T, delta, ie_2goods)
results_rg = simulate_2d_model(y0, A, T, delta, rg_2goods)
results_cn = simulate_2d_model(y0, A, T, delta, cn_2goods)

#### Accuracy comparison

See the phase diagram below for the motivation to prefer the Crank-Nicolson method over either of the two Euler schemes. We could show analytically that the system should stay in a stable limit cycle, orbiting the equilibrium. As the Euler schemes are first-order Taylor approximations of the "true" behaviour of the system, they create small errors, when second-order effects are present. Namely, in the case of a convex function ($\frac{\partial^2f}{\partial x^2}>0$), the explicit Euler scheme will create a positive bias, and the implicit Euler scheme a negative one. This can easily be proven with Jensen's inequality. The opposite holds for concave functions. The circle being convex, we can verify that the explicit Euler scheme diverges outward, while the implicit one turns inwards. Taking the mean of both will hence always provide much more accurate solutions! (Note: in physical applications, this is often related to the conservation of quantities such as energy.) The Runge method would in this case yield practically the same result as the Crank-Nicolson method, so plotting only one of the two makes more sense here.

In [ ]:
plt.figure(figsize=(6,6))
plt.title("Comparison of timestepping schemes\n in terms of accuracy")

plt.plot(results_ee[0], results_ee[1], c="blue", lw=0.7, label="Explicit Euler")
plt.plot(results_ie[0], results_ie[1], c="red", lw=0.7, label="Implicit Euler")
plt.plot(results_cn[0], results_cn[1], c="black", lw=0.7, label="Crank-Nicolson")

# indicate direction using arrows on the last step
plt.arrow(
    results_ee[0, -1], 
    results_ee[1, -1], 
    results_ee[0, -1] - results_ee[0, -2], 
    results_ee[1, -1] - results_ee[1, -2],
    head_width=0.25, color="blue", lw=0.4,
    length_includes_head=True
)
plt.arrow(
    results_ie[0, -1], 
    results_ie[1, -1], 
    results_ie[0, -1] - results_ie[0, -2], 
    results_ie[1, -1] - results_ie[1, -2],
    head_width=0.25, color="red", lw=0.4,
    length_includes_head=True
)
plt.arrow(
    results_cn[0, -1], 
    results_cn[1, -1], 
    results_cn[0, -1] - results_cn[0, -2], 
    results_cn[1, -1] - results_cn[1, -2],
    head_width=0.25, color="black", lw=0.4,
    length_includes_head=True
)

plt.xlabel("y", fontsize=12)
plt.ylabel("y'", fontsize=12)

plt.legend()